# single class


In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
tqdm.pandas()

class NumpyEncoder(json.JSONEncoder):
    """ 
    https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable
    Special json encoder for numpy types
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

class COCOConverter:
    """Class to convert competition csv to coco format."""
    def __init__(
        self,
        df: pd.DataFrame, 
        image_height: int = 720, 
        image_width: int = 1280, 
        type_agnostic: bool = False):
        
        self.image_height = image_height
        self.image_width = image_width
        self.type_agnostic = type_agnostic
        if self.type_agnostic:
            self.categories = [{"id": 1, "name": "Helmet"}]
        else:
            self.categories = [
                {"id": 1, "name": "impact_None",},
                {"id": 2, "name": "impact_Helmet"},
                {"id": 3, "name": "impact_Shoulder",},
                {"id": 4, "name": "impact_Body"},
                {"id": 5, "name": "impact_Ground",},
                {"id": 6, "name": "impact_Hand"},
            ]         
        self.df = self._initialize(df)

    def _get_file_name(self, row: pd.Series):
        base_name = row.video[:-4]
        file_name = f'{base_name}_frame{row.frame:04}.jpg'
        return file_name

    def _get_bbox(self, row: pd.Series):
        return [row.left, row.top, row.width, row.height]

    def _initialize(self, df: pd.DataFrame):
        # set category id
        if self.type_agnostic:
            df['impactType'] = 'Helmet'
            df['category_id'] = 1
        else:
            df['category_id'] = df['impactType'].map(
                {
                    'None': 1,
                    'Helmet': 2,
                    'Shoulder': 3,
                    'Body': 4,
                    'Ground': 5,
                    'Hand': 6
                }
            )
        # some preprocesses
        df['file_name'] = df[['video', 'frame']].progress_apply(self._get_file_name, axis=1)
        df['area'] = df['width'] * df['height']
        df['bbox'] = df[['left', 'top', 'height', 'width']].progress_apply(self._get_bbox, axis=1)
        df['iscrowd'] = 0
        return df
        

    def save(self, save_path):
        """
        Save as coco json format.
        But also has many supplemental items like gameKey or view.
        """
        df = self.df.copy()
        image_df = df[['gameKey', 'playID', 'view', 'video', 'frame', 'file_name']].drop_duplicates()
        image_df['height'] = self.image_height
        image_df['width'] = self.image_width
        
        # add image id to images. Note that it's called just "id".
        image_df['id'] = range(1, len(image_df) + 1)
    
        # add image id to annotations.
        df['image_id'] = df[['file_name']].merge(image_df[['file_name', 'id']])['id'].values
        df['id'] = range(1, len(df) + 1)

        print('start dumping...')
        coco_annotations = dict()
        coco_annotations['categories'] = self.categories
        coco_annotations['images'] = [dict(row) for _, row in image_df.iterrows()]
        coco_annotations['annotations'] = [dict(row) for _, row in df.iterrows()]
        json.dump(coco_annotations, open(save_path, 'w'), indent=4, cls=NumpyEncoder)

In [10]:
df = pd.read_csv('work/NFL/train_labels.csv')

In [11]:
play_ids = df['playID'].unique()
num_train = int(len(play_ids) * 0.8)
train_play_ids = df['playID'].unique()[:num_train]
valid_play_ids = df['playID'].unique()[num_train:]
print('number of train videos:', len(train_play_ids))
print('number of valid videos:', len(valid_play_ids))

number of train videos: 48
number of valid videos: 12


In [12]:
train_df = df.query('playID in @train_play_ids').reset_index(drop=True).copy()
valid_df = df.query('playID in @valid_play_ids').reset_index(drop=True).copy()

print('number of train annotations:', len(train_df))
print('number of valid annotations:', len(valid_df))

number of train annotations: 753170
number of valid annotations: 198917


In [13]:
train_coco = COCOConverter(train_df, type_agnostic=True)
train_coco.save('train_single.json')
valid_coco = COCOConverter(valid_df, type_agnostic=True)
valid_coco.save('valid_single.json')

100%|██████████| 753170/753170 [00:28<00:00, 26696.18it/s]


start dumping...


100%|██████████| 198917/198917 [00:07<00:00, 26484.87it/s]


start dumping...


# multiclass

In [3]:
import random
import numpy as np
from pathlib import Path
import datetime
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import cv2
import os
import json
import matplotlib.pyplot as plt
from IPython.core.display import Video, display
import subprocess
import gc
import shutil

In [4]:
import pandas as pd

# Load image level csv file
extra_df = pd.read_csv('./work/NFL/image_labels.csv')
print('Number of ground truth bounding boxes: ', len(extra_df))

# Number of unique labels
label_to_id = {label: i for i, label in enumerate(extra_df.label.unique())}
print('Unique labels: ', label_to_id)

Number of ground truth bounding boxes:  193736
Unique labels:  {'Helmet': 0, 'Helmet-Blurred': 1, 'Helmet-Difficult': 2, 'Helmet-Sideline': 3, 'Helmet-Partial': 4}


In [5]:
def create_ann_file(df, category_id):
    
    now = datetime.datetime.now()

    data = dict(
        info=dict(
            description='NFL-Helmet-Assignment',
            url=None,
            version=None,
            year=now.year,
            contributor=None,
            date_created=now.strftime('%Y-%m-%d %H:%M:%S.%f'),
        ),
        licenses=[dict(
            url=None,
            id=0,
            name=None,
        )],
        images=[
            # license, url, file_name, height, width, date_captured, id
        ],
        type='instances',
        annotations=[
            # segmentation, area, iscrowd, image_id, bbox, category_id, id
        ],
        categories=[
            # supercategory, id, name
        ],
    )
    
    class_name_to_id = {}
    labels =  ["__ignore__",
                'Helmet',
              'Helmet-Blurred', 
              'Helmet-Difficult', 
              'Helmet-Sideline',
              'Helmet-Partial']

    for i, each_label in enumerate(labels):
        class_id = i - 1  # starts with -1
        class_name = each_label
        if class_id == -1:
            assert class_name == '__ignore__'
            continue
        class_name_to_id[class_name] = class_id
        data['categories'].append(dict(
            supercategory=None,
            id=class_id,
            name=class_name,
        ))
    
    box_id = 0
    for i, image in enumerate(os.listdir(TRAIN_PATH)):

        img = cv2.imread(TRAIN_PATH+'/'+image)
        height, width, _ = img.shape

        data['images'].append({
            'license':0, 
            'url': None,
            'file_name': image,
            'height': height,
            'width': width,
            'date_camputured': None,
            'id': i
        })

        df_temp = df[df.image == image]
        for index, row in df_temp.iterrows():

            area = round(row.width*row.height, 1)
            bbox =[row.left, row.top, row.width, row.height]

            data['annotations'].append({
                'id': box_id,
                'image_id': i,
                'category_id': category_id[row.label],
                'area': area,
                'bbox':bbox,
                'iscrowd':0
            })
            box_id+=1
    
    return data

In [10]:
from sklearn.model_selection import train_test_split

TRAIN_PATH = './work/NFL/images'
extra_df = pd.read_csv('./work/NFL/image_labels.csv')

category_id = {'Helmet':0, 'Helmet-Blurred':1,
               'Helmet-Difficult':2, 'Helmet-Sideline':3,
               'Helmet-Partial':4}

df_train, df_val = train_test_split(extra_df, test_size=0.2, random_state=66)
print('train:',df_train.shape,'val:',df_val.shape)

train: (154988, 6) val: (38748, 6)


In [ ]:
image_bbox_label = {} 
for image, df in extra_df.groupby('image'): 
    image_bbox_label[image] = df.reset_index(drop=True)
train_names, valid_names = train_test_split(list(image_bbox_label), test_size=0.2, random_state=42)
print(f'Size of dataset: {len(image_bbox_label)},\
       training images: {len(train_names)},\
       validation images: {len(valid_names)}')


Size of dataset: 9947,       training images: 7957,       validation images: 1990


In [ ]:
frames=[]
for i in train_names:
    frames.append(extra_df[extra_df['image']==i])
df_train = pd.concat(frames)


frames=[]
for i in valid_names:
    frames.append(extra_df[extra_df['image']==i])
df_val = pd.concat(frames)

In [ ]:
print(df_val.head())

In [ ]:
print(extra_df.head())

In [ ]:
ann_file_train = create_ann_file(df_train.reset_index(), category_id)
ann_file_val = create_ann_file(df_val.reset_index(), category_id)

In [ ]:
print('train:',df_train.shape,'val:',df_val.shape)

In [ ]:
#save as json to gdrive
with open('work/NFL/ann_file_train.json', 'w') as f:
    json.dump(ann_file_train, f, indent=4)
        
with open('work/NFL/ann_file_val.json', 'w') as f:
    json.dump(ann_file_val, f, indent=4)

In [81]:
import base64
import IPython
import json
import numpy as np
import os
import random
import requests
from io import BytesIO
from math import trunc
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

In [82]:
class CocoDataset():
    def __init__(self, annotation_path, image_dir):
        self.annotation_path = annotation_path
        self.image_dir = image_dir
        self.colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                        'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                        'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                        'magenta', 'sienna', 'maroon']
        
        json_file = open(self.annotation_path)
        self.coco = json.load(json_file)
        json_file.close()
        
        #self.process_info()
        #self.process_licenses()
        self.process_categories()
        self.process_images()
        self.process_segmentations()

    def display_info(self):
        print('Dataset Info:')
        print('=============')
        for key, item in self.info.items():
            print('  {}: {}'.format(key, item))
        
        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        print('')
        
    def display_licenses(self):
        print('Licenses:')
        print('=========')
        
        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(req, str(req_type)))
            print('')
        print('')
        
    def display_categories(self):
        print('Categories:')
        print('=========')
        for sc_key, sc_val in self.super_categories.items():
            print('  super_category: {}'.format(sc_key))
            for cat_id in sc_val:
                print('    id {}: {}'.format(cat_id, self.categories[cat_id]['name']))
            print('')
    
    def display_image(self, image_id, show_polys=False, show_bbox=True, show_labels=True, show_crowds=False, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))
        
        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))
            
        # Open the image
        if use_url:
            image_path = image['coco_url']
            response = requests.get(image_path)
            image = PILImage.open(BytesIO(response.content))
            
        else:
            image_path = os.path.join(self.image_dir, image['file_name'])
            image = PILImage.open(image_path)
            
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = "data:image/png;base64, " + base64.b64encode(buffered.getvalue()).decode()
        
        # Calculate the size and adjusted display size
        max_width = 900
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height
        
        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        labels = {}
        print('  segmentations ({}):'.format(len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            polygons_list = []
            if segm['iscrowd'] != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height) * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1 , 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append({'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})
                            
                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1 # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(full_lines_to_insert * adjusted_ratio)
                                rle_list.append({'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})
                                
                            # Insert bottom line
                            rle_list.append({'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list  
            # else:
            #     # Add the polygon segmentation
            #     for segmentation_points in segm['segmentation']:
            #         segmentation_points = np.multiply(segmentation_points, adjusted_ratio).astype(int)
            #         polygons_list.append(str(segmentation_points).lstrip('[').rstrip(']'))

            polygons[segm['id']] = polygons_list

            if i < len(self.colors):
                poly_colors[segm['id']] = self.colors[i]
            else:
                poly_colors[segm['id']] = 'white'
            
            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] + bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[segm['id']] = str(bbox_points).lstrip('[').rstrip(']')
            
            labels[segm['id']] = (self.categories[segm['category_id']]['name'], (bbox_points[0], bbox_points[1] - 4))
            
            # Print details
            print('    {}:{}:{}'.format(segm['id'], poly_colors[segm['id']], self.categories[segm['category_id']]))

        # Draw segmentation polygons on image
        html = '<div class="container" style="position:relative;">'
        html += '<img src="{}" style="position:relative;top:0px;left:0px;width:{}px;">'.format(img_str, adjusted_width)
        html += '<div class="svgclass"><svg width="{}" height="{}">'.format(adjusted_width, adjusted_height)
        
        if show_polys:
            for seg_id, points_list in polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for points in points_list:
                    html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5" />'.format(points, fill_color, stroke_color)
        
        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(x, y, w, h, fill_color, stroke_color)
            
        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(points, fill_color, stroke_color)
                
        if show_labels:
            for seg_id, label in labels.items():
                color = poly_colors[seg_id]
                html += '<text x="{}" y="{}" style="fill:{}; font-size: 12pt;">{}</text>'.format(label[1][0], label[1][1], color, label[0])
                
        html += '</svg></div>'
        html += '</div>'
        html += '<style>'
        html += '.svgclass { position:absolute; top:0px; left:0px;}'
        html += '</style>'
        return html
       
    def process_info(self):
        self.info = self.coco['info']
    
    def process_licenses(self):
        self.licenses = self.coco['licenses']
    
    def process_categories(self):
        self.categories = {}
        self.super_categories = {}
        for category in self.coco['categories']:
            cat_id = category['id']
            super_category = category['supercategory']
            
            # Add category to the categories dict
            if cat_id not in self.categories:
                self.categories[cat_id] = category
            else:
                print("ERROR: Skipping duplicate category id: {}".format(category))

            # Add category to super_categories dict
            if super_category not in self.super_categories:
                self.super_categories[super_category] = {cat_id} # Create a new set with the category id
            else:
                self.super_categories[super_category] |= {cat_id} # Add category id to the set
                
    def process_images(self):
        self.images = {}
        for image in self.coco['images']:
            image_id = image['id']
            if image_id in self.images:
                print("ERROR: Skipping duplicate image id: {}".format(image))
            else:
                self.images[image_id] = image
                
    def process_segmentations(self):
        self.segmentations = {}
        for segmentation in self.coco['annotations']:
            image_id = segmentation['image_id']
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append(segmentation)

In [71]:
annotation_path = r'work/NFL/ann_file_train.json'
image_dir = r'work/NFL/images'

coco_dataset = CocoDataset(annotation_path, image_dir)
# coco_dataset.display_info()
# coco_dataset.display_licenses()
coco_dataset.display_categories()

Categories:
  super_category: None
    id 0: Helmet
    id 1: Helmet-Blurred
    id 2: Helmet-Difficult
    id 3: Helmet-Sideline
    id 4: Helmet-Partial



In [72]:
html = coco_dataset.display_image('random', use_url=False)
IPython.display.HTML(html)

Image:
  license: 0
  url: None
  file_name: 57873_000399_Endzone_frame1010.jpg
  height: 720
  width: 1280
  date_camputured: None
  id: 2066


KeyError: 2066